In [1]:
%load_ext autoreload

%autoreload 2

from enum import auto, StrEnum

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import streamlit as st



In [2]:
from datetime import datetime
from enum import auto, StrEnum
import re

from requests_cache import CachedSession

from urllib.parse import quote
from urllib.request import urlopen

import pandas as pd
import matplotlib.pyplot as plt


plt.rcParams["figure.figsize"] = (10, 10)


class Preset(StrEnum):
    duel = auto()
    team = auto()
    ffa = auto()
    all = auto()


# Cache the data
def get_data(url: str):
    # print(f"Getting data from {url}")
    session = CachedSession("bar_cache", backend="sqlite")
    data = session.get(url).json()
    return data


def get_fresh_data(url: str):
    # print(f"Getting data from {url}")
    session = CachedSession("short_cache", backend="sqlite", expire_after=60)
    data = session.get(url).json()
    return data


# Get user name from user id
def get_user_name(user_id: int):
    name_list = get_data("https://api.bar-rts.com/cached-users")
    user_name = ""
    for user in name_list:
        if user["id"] == user_id:
            user_name = user["username"]
    return user_name


def get_user_id(user_name: str):
    name_list = get_data("https://api.bar-rts.com/cached-users")
    user_id = ""
    for user in name_list:
        if user["username"] == user_name:
            user_id = user["id"]
    return user_id


def process_match_data(match_details_df):
    if match_details_df.empty:
        return match_details_df
    # Get the winning team and count the number of wins
    match = {}
    matches = []

    for _, game in match_details_df.iterrows():
        # print(game)
        for team in game["AllyTeams"]:
            for player in team["Players"]:
                match = {
                    **match,
                    **{
                        "id": team["id"],
                        "userId": player["userId"],
                        "teamId": player["teamId"],
                        "allyTeamId": player["allyTeamId"],
                        "name": player["name"],
                        "faction": player["faction"],
                        "rank": player["rank"],
                        "skillUncertainty": player["skillUncertainty"],
                        "skill": float(re.sub("[^0123456789\.]", "", player["skill"])),
                        "startPos": player["startPos"],
                        "winningTeam": team["winningTeam"],
                        "Map.fileName": game["Map.fileName"],
                        # + "_"
                        # + str(len(team["Players"]))
                        # + "v"
                        # + str(len(team["Players"])),
                        "Map.scriptName": game["Map.scriptName"],
                        "durationMs": game["durationMs"],
                        "startTime": game["startTime"],
                    },
                }
                matches.append(match)

    matches_df = pd.json_normalize(matches)
    matches_df["startTime"] = pd.to_datetime(matches_df["startTime"])
    return matches_df


# Get match details for each match
def get_match_details(id: str):
    match_details = get_data(f"https://api.bar-rts.com/replays/{id}")

    match_details_df = pd.json_normalize(match_details)
    match_details_df["startTime"] = pd.to_datetime(match_details_df["startTime"])
    return match_details_df


# Get the players replays metadata
def get_match_data(
    matches_bar,
    user: str,
    preset: Preset = Preset.team,
    season0: bool = False,
):
    # Depending on the preset, the API returns different data.json
    # The preset uses the following format: &preset=duel%2Cffa%2Cteam
    # duel%2Cffa%2Cteam is the same as duel,ffa,team

    if preset == Preset.all:
        preset = "&preset=duel&preset=ffa&preset=team"
    else:
        preset = f"&preset={preset.name}"

    # If season0 is true, then only get games after June 1st, 2023 %2C
    date_range = ""
    if season0:
        date_range = f"&date=2023-06-01&date={datetime.today().strftime('%Y-%m-%d')}"

    uri = f"https://api.bar-rts.com/replays?page=1&limit=9999{preset}{date_range}&hasBots=false&endedNormally=true&players="

    data = get_data(f"{uri}{quote(user)}")

    # Test data has attribute data
    # if not hasattr(data, "data"):
    #     return pd.DataFrame()

    # Get the winning team and count the number of wins
    matches = []
    percent_complete = 0
    number_of_matches = len(data["data"])
    for index, game in enumerate(data["data"]):
        percent_complete += 1
        matches_bar.progress(
            percent_complete / number_of_matches,
            text=f"Getting API for game {index+1} out of {number_of_matches} games. ({game['id']})",
        )
        if game["Map"]["fileName"] is not None:
            matches.append(get_match_details(game["id"]))

    if len(matches) == 0:
        return pd.DataFrame()

    matches_df = pd.concat(matches, axis=0)
    return matches_df


# Get the win rate for each map
def get_win_rate(
    df,
    user: str,
    min_games: int = 5,
):
    if df.empty:
        return df

    user_id = get_user_id(user)
    if user_id == "":
        print(f"{user} does not exist")
        return pd.DataFrame()

    win_rate_df = (
        df.query(f"userId == {user_id}")
        .groupby(["Map.fileName"])
        .agg({"winningTeam": ["mean", "count"]})["winningTeam"]
        .query(f"count >= {str(min_games)}")
        .sort_values([("mean"), ("count")], ascending=True)
    )
    return win_rate_df


def get_fractions_win_rate(df, user: str):
    return (
        df.query(f"userId == {get_user_id(user)}")
        .groupby(["faction"])
        .agg({"winningTeam": ["mean", "count"]})["winningTeam"]
    )


def get_best_teammates(df, user: str, min_games: int = 5):
    team_winrate = df.groupby(  # .query(f"userId == {get_user_id('furyhawk')}")
        ["allyTeamId", "userId"]
    ).agg({"winningTeam": ["mean", "count"]})["winningTeam"]

    # From team_winrate, get the allyTeamId of the user
    team_ally_id = team_winrate.query(f"userId == {get_user_id(user)}").index.unique(
        level="allyTeamId"
    )
    teams_df = df[df["allyTeamId"].isin(team_ally_id)]
    best_teammates_df = (
        teams_df.groupby(["name", "userId"])
        .agg({"winningTeam": ["mean", "count"]})["winningTeam"]
        .query(f"count >= {str(min_games)} & userId != {get_user_id(user)}")
        .sort_values([("mean"), ("count")], ascending=False)
    )
    return best_teammates_df


def get_battle_list():
    battles_json = get_fresh_data("https://api.bar-rts.com/battles")
    return pd.json_normalize(battles_json)


def get_battle_details(battles_df):
    battle_list = []

    best_battle = battles_df.head(1)
    for players in best_battle["players"]:
        for player in players:
            if "teamId" in player:  # skill userId
                battle_list.append(
                    {
                        "teamId": player["teamId"],
                        "username": player["username"],
                        "userId": player["userId"],
                        "skill": float(re.sub("[^0123456789\.]", "", player["skill"])),
                        "gameStatus": player["gameStatus"],
                        "map": best_battle["mapFileName"].values[0],
                        "title": best_battle["title"].values[0],
                    }
                )

    return pd.DataFrame(battle_list).sort_values(by="teamId")


def get_map_win_rate(win_rate_df, map_name: str):
    win_rate_df = win_rate_df.reset_index()
    return win_rate_df.loc[win_rate_df["Map.fileName"] == map_name]


In [3]:

plt.rcParams["figure.figsize"] = (10, 10)


class Preset(StrEnum):
    """Preset for the API call"""

    duel = auto()
    team = auto()
    ffa = auto()
    all = auto()


# Set the x axis minor locator to 5 and major locator to 10
# Set the y axis to the map name
def plot_win_rate(
    win_rate_df,
    user: str,
    preset=Preset.team,
):
    # Get overall win rate
    overall_win_rate = win_rate_df["mean"].mean()
    # Get total number of games
    total_games = win_rate_df["count"].sum()

    fig, (ax1, ax2) = plt.subplots(1, 2, sharey="all", figsize=(12, 6))

    bars = ax1.barh(
        y=win_rate_df.index,
        width=win_rate_df["mean"],
        alpha=0.75,
    )
    ax1.bar_label(bars, fmt="{:.0%}", label_type="center")
    ax1.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))
    ax1.xaxis.set_major_locator(ticker.MultipleLocator(0.1))
    ax1.grid(which="minor", axis="x", linestyle="--")
    ax1.grid(which="major", axis="x", linestyle="-")
    ax1.set_xlabel("Winning %")
    ax1.set_ylabel("Map")
    ax1.set_xlim(0, 1)
    ax1.set_title(f"[{preset.name} games]{user} Winning {overall_win_rate:.0%} by Map")

    ax1.set_axisbelow(True)

    bars = ax2.barh(
        y=win_rate_df.index,
        width=win_rate_df["count"],
        alpha=0.75,
    )

    ax2.bar_label(bars, label_type="center")
    ax2.xaxis.set_minor_locator(ticker.MultipleLocator(5))
    ax2.xaxis.set_major_locator(ticker.MultipleLocator(10))
    ax2.grid(which="minor", axis="x", linestyle="--")
    ax2.grid(which="major", axis="x", linestyle="-")
    ax2.set_xlabel("Games")
    ax2.set_title(f"{user} {total_games} games by Map")

    plt.subplots_adjust(wspace=0, hspace=0)

    return fig

In [5]:
preset = Preset.team
season0: bool = True
user = "furyhawk"

In [7]:
progress_text = "Operation in progress. Please wait."
matches_bar = st.progress(0, text=progress_text)

2023-07-09 18:39:27.083 
  command:

    streamlit run /opt/homebrew/Caskroom/miniforge/base/envs/web/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [8]:
df = process_match_data(get_match_data(matches_bar, user, preset, season0))

In [9]:
df

,id,userId,teamId,allyTeamId,name,faction,rank,skillUncertainty,skill,winningTeam,Map.fileName,Map.scriptName,durationMs,startTime,startPos.x,startPos.y,startPos.z
0,715941,4040,5,715941,Karunel,Armada,3,3,26.74,False,flats_and_forests_v2.1,Flats and Forests v2.1,1004467,2023-07-06 13:26:31+00:00,11646.758789,217.387939,3440.973145
1,715941,101939,4,715941,Litschi,Armada,5,3,33.98,False,flats_and_forests_v2.1,Flats and Forests v2.1,1004467,2023-07-06 13:26:31+00:00,10197.106445,207.090332,7777.133789
2,715941,1256,2,715941,sneyed,Armada,5,3,38.58,False,flats_and_forests_v2.1,Flats and Forests v2.1,1004467,2023-07-06 13:26:31+00:00,11631.319336,224.760742,8888.593750
3,715941,53682,3,715941,fiddler112,Cortex,5,3,35.35,False,flats_and_forests_v2.1,Flats and Forests v2.1,1004467,2023-07-06 13:26:31+00:00,11715.701172,210.979980,11950.851562
4,715941,92112,6,715941,BattleMoose,Armada,3,3,25.46,False,flats_and_forests_v2.1,Flats and Forests v2.1,1004467,2023-07-06 13:26:31+00:00,10247.027344,225.524170,1518.017456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,616367,96033,12,616367,lolfindkeinnamen,Cortex,2,3,16.17,True,supreme_strait_v1.4,Supreme Strait v1.4,2066500,2023-06-01 14:28:39+00:00,4535.524902,59.813599,7434.944824
723,616367,83880,13,616367,Ambassador,Armada,3,3,14.79,True,supreme_strait_v1.4,Supreme Strait v1.4,2066500,2023-06-01 14:28:39+00:00,3944.423584,62.832031,9278.013672
724,616367,88793,10,616367,ThePasst,Armada,3,3,18.00,True,supreme_strait_v1.4,Supreme Strait v1.4,2066500,2023-06-01 14:28:39+00:00,972.909546,265.818604,10482.109375
725,616367,19953,9,616367,PaniniPie,Cortex,5,3,20.45,True,supreme_strait_v1.4,Supreme Strait v1.4,2066500,2023-06-01 14:28:39+00:00,4863.265137,55.085083,8604.011719


In [17]:
# Get the players replays metadata
def get_quick_match_data(
    user: str,
    preset: Preset = Preset.team,
    season0: bool = False,
):
    # Depending on the preset, the API returns different data.json
    # The preset uses the following format: &preset=duel%2Cffa%2Cteam
    # duel%2Cffa%2Cteam is the same as duel,ffa,team

    if preset == Preset.all:
        preset = "&preset=duel&preset=ffa&preset=team"
    else:
        preset = f"&preset={preset.name}"

    # If season0 is true, then only get games after June 1st, 2023 %2C
    date_range = ""
    if season0:
        date_range = f"&date=2023-06-01&date={datetime.today().strftime('%Y-%m-%d')}"

    uri = f"https://api.bar-rts.com/replays?page=1&limit=9999{preset}{date_range}&hasBots=false&endedNormally=true&players="

    data = get_fresh_data(f"{uri}{quote(user)}")

    # Get the winning team and count the number of wins
    match = {}
    matches = []

    for game in data["data"]:
        if game["Map"]["fileName"] is not None:
            # print(game["id"])
            for team in game["AllyTeams"]:
                for player in team["Players"]:
                    if player["name"] == user:
                        match = {
                            **match,
                            **{
                                "id": game["id"],
                                "name": player["name"],
                                "winningTeam": team["winningTeam"],
                                "Map.fileName": game["Map"]["fileName"],
                                "Map.scriptName": game["Map"]["scriptName"],
                                "durationMs": game["durationMs"],
                                "startTime": game["startTime"],
                            },
                        }
                        matches.append(match)

    matches_df = pd.json_normalize(matches)
    matches_df["startTime"] = pd.to_datetime(matches_df["startTime"])
    return matches_df


In [18]:
win_rate_df = get_quick_match_data(user, preset)

In [23]:
# Get the win rate for each map
def get_quick_win_rate(
    df,
    user: str,
    min_games: int = 5,
):
    if df.empty:
        return df

    win_rate_df = (
        df.groupby(["Map.fileName"])
        .agg({"winningTeam": ["mean", "count"]})["winningTeam"]
        .query(f"count >= {str(min_games)}")
        .sort_values([("mean"), ("count")], ascending=True)
    )
    return win_rate_df


In [19]:
win_rate_df

,id,name,winningTeam,Map.fileName,Map.scriptName,durationMs,startTime
0,07c1a664c27e2db84d32317adddb0d8f,furyhawk,True,flats_and_forests_v2.1,Flats and Forests v2.1,1004467,2023-07-06 13:26:31+00:00
1,dd8fa664a7c8cd36edc3fe41334950a8,furyhawk,True,dsdr_4.1,DSDR 4.1,881800,2023-07-06 09:56:45+00:00
2,2ddba064272bb2999331ae9156d67396,furyhawk,True,ascendancy_v2.0,Ascendancy v2.0,1184533,2023-07-02 02:04:29+00:00
3,49e29e64f77aa4e8e5faf054124d74d2,furyhawk,True,ice_scream_v2.5,Ice Scream v2.5,3308000,2023-06-30 14:10:17+00:00
4,14bd9e6476a5c8778c995d2881393fb6,furyhawk,True,dsdr_4.1,DSDR 4.1,2427033,2023-06-30 11:31:32+00:00
...,...,...,...,...,...,...,...
106,155a5064a582dc362ae4ac708e074e2a,furyhawk,True,ice_scream_v2.5,Ice Scream v2.5,2213167,2023-05-02 00:32:21+00:00
107,b7d24f642113204e8d20d0ab54ad5044,furyhawk,False,supreme_strait_v1.4,Supreme Strait v1.4,1197000,2023-05-01 14:54:47+00:00
108,67c54f643202a0027285f45ae3cf6736,furyhawk,False,thermal_shock_v1.1,Thermal Shock v1.1,1409767,2023-05-01 13:57:59+00:00
109,77b84f649b657130c907ce491b4fbef0,furyhawk,True,dsdr_4.1,DSDR 4.1,2292467,2023-05-01 13:02:47+00:00


In [34]:
get_quick_win_rate(win_rate_df, user, 5).to_dict("records")

[{'mean': 0.3333333333333333, 'count': 6},
 {'mean': 0.3333333333333333, 'count': 6},
 {'mean': 0.4, 'count': 5},
 {'mean': 0.42857142857142855, 'count': 7},
 {'mean': 0.5, 'count': 10},
 {'mean': 0.6052631578947368, 'count': 38},
 {'mean': 0.7, 'count': 10}]

In [38]:
win_rate_user_df = get_quick_win_rate(win_rate_df, user, 5)
df = win_rate_user_df.reset_index()
map_df = df.loc[df["Map.fileName"] == player["map"]]
records = map_df.to_dict("records")[-1]

In [39]:
records

{'Map.fileName': 'supreme_strait_v1.4',
 'mean': 0.6052631578947368,
 'count': 38}

In [27]:
battles_df = get_battle_list()
battle_detail_df = get_battle_details(battles_df)
team_size = len(battle_detail_df.index)
battle_detail_df.head(team_size // 2)
battle_detail_df.tail(team_size // 2)

,teamId,username,userId,skill,gameStatus,map,title
7,9,markivs,42091,39.24,Waiting,supreme_strait_v1.4,EU - 3 08
9,10,DangerosoDavo,77397,34.74,Waiting,supreme_strait_v1.4,EU - 3 08
6,11,XeroRenegade,77781,33.69,Waiting,supreme_strait_v1.4,EU - 3 08
12,12,tracker10,94669,28.20,Waiting,supreme_strait_v1.4,EU - 3 08
0,13,adzii,90628,24.16,Waiting,supreme_strait_v1.4,EU - 3 08
14,14,frequency24,68279,21.41,Waiting,supreme_strait_v1.4,EU - 3 08
4,15,TheShorkk,58513,19.26,Waiting,supreme_strait_v1.4,EU - 3 08
8,16,M3LL,124041,14.81,Waiting,supreme_strait_v1.4,EU - 3 08


In [28]:
battle_detail_df

,teamId,username,userId,skill,gameStatus,map,title
10,1,Snoopa,94610,45.38,Waiting,supreme_strait_v1.4,EU - 3 08
5,2,EmperorGlass,24840,38.60,Waiting,supreme_strait_v1.4,EU - 3 08
11,3,WeszPL,89905,30.12,Waiting,supreme_strait_v1.4,EU - 3 08
2,4,conceptchaos,46197,25.64,Waiting,supreme_strait_v1.4,EU - 3 08
13,5,HotshotGG,117040,24.96,Waiting,supreme_strait_v1.4,EU - 3 08
1,6,MOdxp,75116,22.76,Waiting,supreme_strait_v1.4,EU - 3 08
3,7,Nat135,10818,21.30,Waiting,supreme_strait_v1.4,EU - 3 08
15,8,T4_ligase,72375,7.43,Waiting,supreme_strait_v1.4,EU - 3 08
7,9,markivs,42091,39.24,Waiting,supreme_strait_v1.4,EU - 3 08
9,10,DangerosoDavo,77397,34.74,Waiting,supreme_strait_v1.4,EU - 3 08


In [51]:
battle_list = []

for _, player in battle_detail_df.iterrows():
    battle = {}

    win_rate_user_df = get_quick_win_rate(win_rate_df, player["username"], 5)
    df = win_rate_user_df.reset_index()
    map_df = df.loc[df["Map.fileName"] == player["map"]]
    record = map_df.to_dict("records")[-1]
    battle = {"username": player["username"], **record}
    # battle = {**battle, **{"username": player["username"], **record}}
    battle_list.append(battle)


In [52]:
battle_list

[{'username': 'Snoopa',
  'Map.fileName': 'supreme_strait_v1.4',
  'mean': 0.6052631578947368,
  'count': 38},
 {'username': 'EmperorGlass',
  'Map.fileName': 'supreme_strait_v1.4',
  'mean': 0.6052631578947368,
  'count': 38},
 {'username': 'WeszPL',
  'Map.fileName': 'supreme_strait_v1.4',
  'mean': 0.6052631578947368,
  'count': 38},
 {'username': 'conceptchaos',
  'Map.fileName': 'supreme_strait_v1.4',
  'mean': 0.6052631578947368,
  'count': 38},
 {'username': 'HotshotGG',
  'Map.fileName': 'supreme_strait_v1.4',
  'mean': 0.6052631578947368,
  'count': 38},
 {'username': 'MOdxp',
  'Map.fileName': 'supreme_strait_v1.4',
  'mean': 0.6052631578947368,
  'count': 38},
 {'username': 'Nat135',
  'Map.fileName': 'supreme_strait_v1.4',
  'mean': 0.6052631578947368,
  'count': 38},
 {'username': 'T4_ligase',
  'Map.fileName': 'supreme_strait_v1.4',
  'mean': 0.6052631578947368,
  'count': 38},
 {'username': 'markivs',
  'Map.fileName': 'supreme_strait_v1.4',
  'mean': 0.6052631578947368,

In [43]:
battle_win_rate_df = pd.concat(battle_list, axis=0)

TypeError: cannot concatenate object of type '<class 'dict'>'; only Series and DataFrame objs are valid

In [30]:
win_rate_user_df

,mean,count
Map.fileName,,
Rosetta1.4,0.333333,6
koom_valley_3_3.1,0.333333,6
all_that_glitters_v2.1,0.400000,5
ice_scream_v2.5,0.428571,7
all_that_glitters_v2.0,0.500000,10
supreme_strait_v1.4,0.605263,38
dsdr_4.1,0.700000,10
